In [31]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

#change n
n = 27

try :
    scope.dis()
    target.dis()
except Exception as e:
    pass

scope = cw.scope()
target = cw.target(scope)

scope.default_setup()

cw.program_target(scope, cw.programmers.STM32FProgrammer, "./fizzy-CWNANO-true.hex")

scope.arm()

scope.adc.samples = 100000

def read_csv(file_name):
    from numpy import genfromtxt
    return genfromtxt(file_name, delimiter=',')

def save_csv(file_name, trace):
    import numpy as np
    np.savetxt(file_name, trace, delimiter=",")


In [15]:
import time
import numpy as np
scope.adc.samples = 100000
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('s', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace(as_int=True)
    return np.array(trace)

In [16]:
%matplotlib tk
trace = np.array(cap_pass_trace(bytearray([])))
# plt.plot(trace)
# plt.show()

In [17]:
pattern = read_csv('Pat_final.csv')

In [18]:
from tqdm.notebook import tqdm
# res = {}

temp_list = []
for i in tqdm(range(len(trace)-len_arr)):
    temp = trace[i:i+len_arr]
    cf = np.corrcoef(pattern,temp)[0][1]
    temp_list.append(cf)
    

  0%|          | 0/99990 [00:00<?, ?it/s]

In [102]:
%matplotlib tk
from matplotlib.patches import Rectangle
fig, ax = plt.subplots(2,sharex=True)
ax[0].plot(trace)
for i,ele in enumerate(np.array(temp_list)):
    if ele > 0.96 :
#         print(i)
        rect = Rectangle((i, 0), len(pattern), 256, facecolor='lightgrey')
        ax[0].add_patch(rect)
ax[1].plot(temp_list)
plt.show()

In [19]:
save_csv('True/trace.csv',trace)

In [20]:
swap = 0
l = []
for i,ele in enumerate(np.array(temp_list)):
    if ele > 0.96 :
        l.append(swap>0)
        swap=0
    elif ele > 0.92:
        swap += 1
l.append(swap>0)

In [21]:
len(l)

352

In [22]:
def swapPositions(list, pos1, pos2):
     
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list

def find_arr(is_swap,n):
    is_swap.reverse()
    arr = [i for i in range(n)]
    k = 0
    for i in range(n-1):
        for j in range(i+1,0,-1):
            if is_swap[k] :
                arr = swapPositions(arr,j,j-1)
            k += 1
    return arr
    

In [23]:
per = find_arr(l[1:],n)
# find_arr([0,0,1,1,1,1,0,1,1,1],5)

In [24]:
per

[26,
 10,
 15,
 11,
 16,
 8,
 23,
 12,
 4,
 5,
 2,
 9,
 0,
 17,
 18,
 6,
 13,
 24,
 1,
 22,
 7,
 25,
 19,
 3,
 20,
 21,
 14]

In [25]:
r = target.simpleserial_read('r',n)

In [26]:
print(r)

CWbytearray(b'41 41 43 43 44 45 45 45 47 48 49 49 49 4c 4c 4d 4e 4e 4e 4f 4f 4f 52 53 53 53 54')


In [27]:
reset_target(scope)

In [28]:
msg_fin = bytearray([int(r[i]) for i in per])
print(msg_fin)

CWbytearray(b'54 49 4d 49 4e 47 53 49 44 45 43 48 41 4e 4e 45 4c 53 41 52 45 53 4f 43 4f 4f 4c')


In [29]:
target.simpleserial_write('a',msg_fin)
print(target.simpleserial_read('r',1))

CWbytearray(b'01')


In [30]:
res = ''
for i in msg_fin:
    res+=chr(i)
print(res)

TIMINGSIDECHANNELSARESOCOOL
